# 1917 영화 댓글 Preprocessing

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import math
import time

## Naver 영화 페이지의 '1917'영화 댓글 Crawling

In [2]:
# Chromedriver로 chrome browser 실행
path = 'C:/Users/user/chromedriver_win32/chromedriver'
browser = webdriver.Chrome( path )

# Naver 영화 페이지의 '1917'영화 페이지 이동
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321'
browser.get( url )

# 평점에 더보기 링크 클릭
view_more = browser.find_element_by_xpath( '//*[@id="content"]/div[1]/div[4]/div[5]/div[2]/div[1]/a' )
view_more.click()

browser.switch_to_frame( browser.find_element_by_id( 'pointAfterListIframe' ) )

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use driver.switch_to.frame instead
  del sys.path[0]


In [3]:
# 전체 댓글수에 따른 페이지 계산
html0 = browser.page_source
html1 = BeautifulSoup( html0, 'lxml' )
html2 = html1.find( 'div', { 'class': 'score_total' } ).find( 'strong' ).findChildren( 'em' )[ 0 ].getText()
total_comment = int( html2.replace(',', '') )
total_page = total_comment / 10

print( total_comment )
print( total_page )
print( math.trunc( total_page ) )

3117
311.7
311


In [4]:
movie_comments = []

for page in range( 0, 2):  # math.trunc( total_page ) ): # 전체 페이지를 이동
    time.sleep( 1 ) #1초의 시간 delay
    html0 = browser.page_source #selenuim에서 현재 있는 browser의 페이지 소스 불러오기
    html1 = BeautifulSoup( html0, 'lxml') #bs4를 이용하여 html로 파싱하기
    html2 = html1.find( 'div', { 'class': 'ifr_area basic_ifr' } ) #댓글을 포함하고 있는 tag찾기
    review0 = html2.find( 'div', { 'class': 'score_result' } ).find_all( 'li' ) #각 댓글별로 list화
    
    for i in range( len( review0 ) ): #한페이지 내에서 모든 댓글을 수집하기 위한 반복문
        review = review0[ i ].find( 'div',{ 'class': 'score_reple' } ).find( 'span' ).text.strip() #댓글
        if review not in '관람객': # 불필요한 내용 삭제
            movie_comments.append( review + '\n' )
        
    #다음페이지로 넘어가기
    if page == 0:
        browser.find_elements_by_xpath('//*[@class = "paging"]/div/a')[10].click()
    else:
        browser.find_elements_by_xpath('//*[@class = "paging"]/div/a')[11].click()


- 마지막 페이지가 되면, 반복문을 빠져나옴.
- 마지막 comment가 포함되지 않는다는 문제점

In [5]:
print(movie_comments)

['충무로: 이거 어케하는거냐?\n', '촬영감독의 영혼까지 갈아넣은 마스터피스\n', '오스카 작품상 탔어도 할말 없었을것 같다.\n', '주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....\n', '닥터스트레인지의 판단력이 좋았다.\n', '아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...\n', '솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.\n', '스포일러가 포함된 감상평입니다. 감상평 보기\n', '진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...\n', '솔직히 나는 기생충보다 이영화가 더 재미있었다.\n', '막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!\n', '원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음\n', '스포일러가 포함된 감상평입니다. 감상평 보기\n']


## 파일 저장 / 로드

In [6]:
# 파일 저장
with open('comment_1917.txt', 'w', encoding = 'utf-8') as f:
    for comment in movie_comments:
        f.write(comment + '\n')

In [43]:
# 파일 로드
# 줄바꿈 기호 제거
movie_comments = []
with open('comment_1917.txt', 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines:
        str = line.rstrip('\n')
        if str != '':
            movie_comments.append(str)

print(movie_comments)

['충무로: 이거 어케하는거냐?', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.', '주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....', '닥터스트레인지의 판단력이 좋았다.', '아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...', '솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.', '스포일러가 포함된 감상평입니다. 감상평 보기', '진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...', '솔직히 나는 기생충보다 이영화가 더 재미있었다.', '막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!', '원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음', '스포일러가 포함된 감상평입니다. 감상평 보기']


In [8]:
# 여러 문장을 한 문장(하나의 문자열)으로 변경
# expected string or bytes-like object 에러 방지

movie_str_comment = ''
for comment in movie_comments:
    movie_str_comment += comment

print(movie_str_comment)

충무로: 이거 어케하는거냐?촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....닥터스트레인지의 판단력이 좋았다.아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.스포일러가 포함된 감상평입니다. 감상평 보기진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...솔직히 나는 기생충보다 이영화가 더 재미있었다.막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음스포일러가 포함된 감상평입니다. 감상평 보기


## Cleaning: 특수문자 제거

In [9]:
# 한 문자열에 대해
# 특수문자 제거

import re

r = '[-=+,#/\?:^$.@*\"~&%.!\'|\(\)\[\]\<\>`\']'

movie_str_comment = re.sub(r, '', movie_str_comment)
print(movie_str_comment)

충무로 이거 어케하는거냐촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다주인공을 카메라가 계속 따라가는데 세트장이 엄청 넓은거에 놀랐습니다 직접 1차대전에 참전하는 느낌 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참닥터스트레인지의 판단력이 좋았다아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음 감독의 참신성이 빛을 발함 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이솔직히 연출력만으로 최고의 영화다 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽스포일러가 포함된 감상평입니다 감상평 보기진짜 원테이크 장면 너무 긴거 아님 촬영하기 진짜 힘들었을 듯솔직히 나는 기생충보다 이영화가 더 재미있었다막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음스포일러가 포함된 감상평입니다 감상평 보기


In [44]:
# 리스트에 있는 여러 문장에 대해, 각 문장 단위로
# 특수 문자 제거 

movie_new_comments = []

for comment in movie_comments:
    print(comment + '\n')
    movie_new_comments.append(re.sub(r, '', comment))
    
print(movie_new_comments)

충무로: 이거 어케하는거냐?

촬영감독의 영혼까지 갈아넣은 마스터피스

오스카 작품상 탔어도 할말 없었을것 같다.

주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....

닥터스트레인지의 판단력이 좋았다.

아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...

솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.

스포일러가 포함된 감상평입니다. 감상평 보기

진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...

솔직히 나는 기생충보다 이영화가 더 재미있었다.

막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!

원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음

스포일러가 포함된 감상평입니다. 감상평 보기

['충무로 이거 어케하는거냐', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다', '주인공을 카메라가 계속 따라가는데 세트장이 엄청 넓은거에 놀랐습니다 직접 1차대전에 참전하는 느낌 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참', '닥터스트레인지의 판단력이 좋았다', '아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음 감독의 참신성이 빛을 발함 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이', '솔직히 

## Text Preprocessing
## 토믄화: 형태소 분석

In [45]:
from nltk.tokenize import word_tokenize, sent_tokenize
from konlpy.tag import Okt

In [46]:
okt = Okt()

- 에러
- JPype 버젼 문제, 현재 JAVA-HOME 위치를 인식하지 못하는 문제
    1. JAVA_HOME 설정 후 JPype 설치해야 함
    2. JPype 버젼과 Python 버젼 차이
    3. konlpy 파일 구조 읽는 방법 (문자열 전체 단위로)

In [16]:
# 형태소 분리 (전체 문자열 단위)

morphemes = []

for morphs in okt.morphs(movie_str_comment):
    morphemes.append(morphs)

print(morphemes)

['충무로', '이', '거', '어케', '하는거냐', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '주인공', '을', '카메라', '가', '계속', '따라가는데', '세트', '장이', '엄청', '넓은거에', '놀랐습니다', '직접', '1', '차', '대전', '에', '참전', '하는', '느낌', '이', '때', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '의', '악마', '로', '자식', '들', '다', '전장', '에', '몰리', '고', '더', '최악', '의', '고통', '을', '받을껀데', '참', '닥터스', '트레인', '지', '의', '판단력', '이', '좋았다', '아카데미', '작품', '상', '이야', '작품', '전체', '를', '보는거니', '기생충', '이', '받을수도', '있다고', '보는데', '감독', '상', '은', '1917', '줬어도', '할말', '없음', '감독', '의', '참신', '성', '이', '빛', '을', '발함', '결론', '적', '으로', '아카데미', '작품', '상과', '감독', '상', '을', '1917', '이', '받았어도', '아무', '할말', '없는', '것', '이', '솔직히', '연출', '력', '만으로', '최고', '의', '영화', '다', '마치', '우리', '가', '투명인간', '이', '되서', '그', '들', '을', '지켜보는', '듯', '했다', '마지막', '장면', '은', '영화', '내내', '쉴', '시간', '이', '없었던것에', '대한', '피로', '감', '을', '풀어줘서', '기승', '전결', '도', '완벽', '스포일러', '가', '포함', '된', '감상', '평', '입니다', '감상', '평', '보기', '진짜', '

In [47]:
# 형태소 분리 (리스트 내 각 문자열 단위)

morphemes = []

for comment in movie_new_comments:
    for morphs in okt.morphs(comment):
        morphemes.append(morphs)

print(morphemes)

['충무로', '이', '거', '어케', '하는거냐', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '주인공', '을', '카메라', '가', '계속', '따라가는데', '세트', '장이', '엄청', '넓은거에', '놀랐습니다', '직접', '1', '차', '대전', '에', '참전', '하는', '느낌', '이', '때', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '의', '악마', '로', '자식', '들', '다', '전장', '에', '몰리', '고', '더', '최악', '의', '고통', '을', '받을껀데', '참', '닥터스', '트레인', '지', '의', '판단력', '이', '좋았다', '아카데미', '작품', '상', '이야', '작품', '전체', '를', '보는거니', '기생충', '이', '받을수도', '있다고', '보는데', '감독', '상', '은', '1917', '줬어도', '할말', '없음', '감독', '의', '참신', '성', '이', '빛', '을', '발함', '결론', '적', '으로', '아카데미', '작품', '상과', '감독', '상', '을', '1917', '이', '받았어도', '아무', '할말', '없는', '것', '이', '솔직히', '연출', '력', '만으로', '최고', '의', '영화', '다', '마치', '우리', '가', '투명인간', '이', '되서', '그', '들', '을', '지켜보는', '듯', '했다', '마지막', '장면', '은', '영화', '내내', '쉴', '시간', '이', '없었던것에', '대한', '피로', '감', '을', '풀어줘서', '기승', '전결', '도', '완벽', '스포일러', '가', '포함', '된', '감상', '평', '입니다', '감상', '평', '보기', '진짜', '

In [48]:
# 어간 추출

morphemes = []

for comment in movie_new_comments:
    for morphs in okt.morphs(comment, stem = True):
        morphemes.append(morphs)

print(morphemes)

['충무로', '이', '거', '어케', '하다', '촬영감독', '의', '영혼', '까지', '갈아', '넣다', '마스터피스', '오스카', '작품', '상', '타다', '하다', '없다', '같다', '주인공', '을', '카메라', '가', '계속', '따라가다', '세트', '장이', '엄청', '넓다', '놀라다', '직접', '1', '차', '대전', '에', '참전', '하다', '느낌', '이', '때', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '의', '악마', '로', '자식', '들', '다', '전장', '에', '몰리', '고', '더', '최악', '의', '고통', '을', '받을껀데', '차다', '닥터스', '트레인', '지', '의', '판단력', '이', '좋다', '아카데미', '작품', '상', '이야', '작품', '전체', '를', '보다', '기생충', '이', '받다', '있다', '보다', '감독', '상', '은', '1917', '주다', '하다', '없다', '감독', '의', '참신', '성', '이', '빛', '을', '발함', '결론', '적', '으로', '아카데미', '작품', '상과', '감독', '상', '을', '1917', '이', '받다', '아무', '하다', '없다', '것', '이', '솔직하다', '연출', '력', '만으로', '최고', '의', '영화', '다', '마치', '우리', '가', '투명인간', '이', '되다', '그', '들', '을', '지켜보다', '듯', '하다', '마지막', '장면', '은', '영화', '내내', '쉬다', '시간', '이', '없다', '대한', '피로', '감', '을', '풀다', '기승', '전결', '도', '완벽', '스포일러', '가', '포함', '되다', '감상', '평', '이다', '감상', '평', '보기', '진짜', '원', '테이크', '장면', '너무',

In [49]:
# 정규화

morphemes = []

for comment in movie_new_comments:
    for morphs in okt.morphs(comment, norm = True):
        morphemes.append(morphs)

print(morphemes)

['충무로', '이', '거', '어떻게', '하는거냐', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '주인공', '을', '카메라', '가', '계속', '따라가는데', '세트', '장이', '엄청', '넓은거에', '놀랐습니다', '직접', '1', '차', '대전', '에', '참전', '하는', '느낌', '이', '때', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '의', '악마', '로', '자식', '들', '다', '전장', '에', '몰리', '고', '더', '최악', '의', '고통', '을', '받을껀데', '참', '닥터스', '트레인', '지', '의', '판단력', '이', '좋았다', '아카데미', '작품', '상', '이야', '작품', '전체', '를', '보는거니', '기생충', '이', '받을수도', '있다고', '보는데', '감독', '상', '은', '1917', '줬어도', '할말', '없음', '감독', '의', '참신', '성', '이', '빛', '을', '발함', '결론', '적', '으로', '아카데미', '작품', '상과', '감독', '상', '을', '1917', '이', '받았어도', '아무', '할말', '없는', '것', '이', '솔직히', '연출', '력', '만으로', '최고', '의', '영화', '다', '마치', '우리', '가', '투명인간', '이', '되서', '그', '들', '을', '지켜보는', '듯', '했다', '마지막', '장면', '은', '영화', '내내', '쉴', '시간', '이', '없었던것에', '대한', '피로', '감', '을', '풀어줘서', '기승', '전결', '도', '완벽', '스포일러', '가', '포함', '된', '감상', '평', '입니다', '감상', '평', '보기', '진짜', 

In [50]:
# 명사 분리

morphemes = []

for comment in movie_new_comments:
    for morphs in okt.nouns(comment):
        morphemes.append(morphs)

print(morphemes)

['충무로', '거', '어케', '촬영감독', '영혼', '마스터피스', '오스카', '작품', '주인공', '카메라', '계속', '세트', '직접', '차', '대전', '참전', '느낌', '때', '당시', '군', '히틀러', '극악', '악마', '자식', '전장', '몰리', '더', '최악', '고통', '받을껀데', '닥터스', '트레인', '의', '판단력', '아카데미', '작품', '작품', '전체', '기생충', '감독', '감독', '참신', '빛', '발함', '결론', '아카데미', '작품', '상과', '감독', '이', '아무', '것', '연출', '최고', '영화', '마치', '우리', '투명인간', '그', '듯', '마지막', '장면', '영화', '내내', '시간', '대한', '피로', '감', '기승', '전결', '완벽', '스포일러', '포함', '감상', '평', '감상', '평', '보기', '진짜', '테이크', '장면', '촬영', '진짜', '듯', '나', '기생충', '이영화', '더', '막판', '횡', '병사', '신호', '우르르', '때', '주인공', '혼자', '종', '막', '장면', '진짜', '대박', '무조건', '아이맥스', '테이크', '촬영', '볼때', '후보', '스포일러', '포함', '감상', '평', '감상', '평', '보기']


In [51]:
# 품사 태깅

morphemes = []

for comment in movie_new_comments:
    for morphs in okt.pos(comment, join = True):
        morphemes.append(morphs)

print(morphemes)

['충무로/Noun', '이/Determiner', '거/Noun', '어케/Noun', '하는거냐/Verb', '촬영감독/Noun', '의/Josa', '영혼/Noun', '까지/Josa', '갈아/Adverb', '넣은/Verb', '마스터피스/Noun', '오스카/Noun', '작품/Noun', '상/Suffix', '탔어도/Verb', '할말/Verb', '없었을것/Adjective', '같다/Adjective', '주인공/Noun', '을/Josa', '카메라/Noun', '가/Josa', '계속/Noun', '따라가는데/Verb', '세트/Noun', '장이/Suffix', '엄청/Adverb', '넓은거에/Adjective', '놀랐습니다/Verb', '직접/Noun', '1/Number', '차/Noun', '대전/Noun', '에/Josa', '참전/Noun', '하는/Verb', '느낌/Noun', '이/Determiner', '때/Noun', '당시/Noun', '군/Noun', '인들/Josa', '20년/Number', '뒤/Foreign', '히틀러/Noun', '라는/Josa', '극악/Noun', '의/Josa', '악마/Noun', '로/Josa', '자식/Noun', '들/Suffix', '다/Adverb', '전장/Noun', '에/Josa', '몰리/Noun', '고/Josa', '더/Noun', '최악/Noun', '의/Josa', '고통/Noun', '을/Josa', '받을껀데/Noun', '참/Verb', '닥터스/Noun', '트레인/Noun', '지/Josa', '의/Noun', '판단력/Noun', '이/Josa', '좋았다/Adjective', '아카데미/Noun', '작품/Noun', '상/Suffix', '이야/Josa', '작품/Noun', '전체/Noun', '를/Josa', '보는거니/Verb', '기생충/Noun', '이/Josa', '받을수도/Verb', '있다고/Adjective', '보는데/Ver

In [52]:
# 어절 분리

morphemes = []

for comment in movie_new_comments:
    for morphs in okt.phrases(comment):
        morphemes.append(morphs)

print(morphemes)

['충무로', '충무로 이거', '충무로 이거 어케', '이거', '어케', '촬영감독', '촬영감독의 영혼', '마스터피스', '영혼', '오스카', '오스카 작품상', '작품상', '주인공', '카메라', '계속', '세트장이', '직접', '직접 1차대전', '참전', '참전하는 느낌', '참전하는 느낌 이때당시', '참전하는 느낌 이때당시 군', '20년', '히틀러', '극악', '극악의 악마', '자식들', '전장', '몰리', '더 최악', '더 최악의 고통', '받을껀데', '대전', '느낌', '이때', '당시', '악마', '최악', '고통', '닥터스트레인', '의 판단력', '닥터스', '트레인', '판단력', '아카데미', '아카데미 작품상', '작품전체', '기생충', '감독상', '1917', '감독', '감독의 참신성', '발함', '아카데미 작품상과', '아카데미 작품상과 감독상', '1917이', '아무', '아무 할말 없는 것', '작품상', '작품', '전체', '참신성', '상과', '연출력', '최고', '최고의 영화', '마치', '마치 우리', '투명인간', '그들', '마지막장면', '영화', '영화 내내', '영화 내내 쉴시간', '대한', '대한 피로감', '기승전결', '완벽', '우리', '마지막', '장면', '내내', '시간', '피로', '기승', '전결', '스포일러', '포함', '포함된 감상평', '감상평', '감상평 보기', '감상', '보기', '진짜', '진짜 원테이크', '진짜 원테이크 장면', '촬영', '진짜 힘들었을 듯', '원테이크', '장면', '기생충', '이영화', '막판', '수많은 병사', '신호', '우르르', '우르르 튀어나올때', '우르르 튀어나올때 주인공', '우르르 튀어나올때 주인공 혼자', '우르르 튀어나올때 주인공 혼자 종', '막 달리는 장면', '진짜', '대박', '대박 무조건', '대박 무조건 아이맥스', '병사', '주인공', '혼자', '장면', '무조

## 불용어 처리

In [53]:
# 불용어 처리

stop_words = []

with open('stopwords.txt', 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append(line.rstrip('\n'))

In [54]:
results = []

for comment in movie_new_comments:
    for morphs in okt.morphs(comment):
        if morphs not in stop_words:
            results.append(morphs)

print(results)

['충무로', '거', '어케', '하는거냐', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '주인공', '카메라', '계속', '따라가는데', '세트', '장이', '엄청', '넓은거에', '놀랐습니다', '직접', '1', '차', '대전', '참전', '하는', '느낌', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '악마', '자식', '다', '전장', '몰리', '고', '더', '최악', '고통', '받을껀데', '닥터스', '트레인', '지', '판단력', '좋았다', '아카데미', '작품', '상', '이야', '작품', '전체', '보는거니', '기생충', '받을수도', '있다고', '보는데', '감독', '상', '은', '1917', '줬어도', '할말', '없음', '감독', '참신', '성', '빛', '발함', '결론', '적', '아카데미', '작품', '상과', '감독', '상', '1917', '받았어도', '아무', '할말', '없는', '솔직히', '연출', '력', '만으로', '최고', '영화', '다', '투명인간', '되서', '지켜보는', '듯', '했다', '마지막', '장면', '은', '영화', '내내', '쉴', '없었던것에', '대한', '피로', '감', '풀어줘서', '기승', '전결', '도', '완벽', '스포일러', '포함', '된', '감상', '평', '입니다', '감상', '평', '보기', '진짜', '원', '테이크', '장면', '너무', '긴거', '아님', '촬영', '하기', '진짜', '힘들었을', '듯', '솔직히', '는', '기생충', '보다', '이영화', '더', '재미있었다', '막판', '횡', '수많은', '병사', '신호', '튀어나올', '주인공', '종', '막', '달리는', '장면', '진짜', 'ㅋㅋㅋㅋ', '대박', '무조건',

## 인코딩: Integer Encoding

In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(results)

In [57]:
print(tokenizer.word_index)

{'작품': 1, '상': 2, '감상': 3, '평': 4, '할말': 5, '감독': 6, '장면': 7, '진짜': 8, '주인공': 9, '다': 10, '더': 11, '아카데미': 12, '기생충': 13, '은': 14, '1917': 15, '솔직히': 16, '영화': 17, '듯': 18, '스포일러': 19, '포함': 20, '된': 21, '입니다': 22, '보기': 23, '원': 24, '테이크': 25, '촬영': 26, '충무로': 27, '거': 28, '어케': 29, '하는거냐': 30, '촬영감독': 31, '영혼': 32, '갈아': 33, '넣은': 34, '마스터피스': 35, '오스카': 36, '탔어도': 37, '없었을것': 38, '카메라': 39, '계속': 40, '따라가는데': 41, '세트': 42, '장이': 43, '엄청': 44, '넓은거에': 45, '놀랐습니다': 46, '직접': 47, '1': 48, '차': 49, '대전': 50, '참전': 51, '하는': 52, '느낌': 53, '당시': 54, '군': 55, '인들': 56, '20년': 57, '뒤': 58, '히틀러': 59, '라는': 60, '극악': 61, '악마': 62, '자식': 63, '전장': 64, '몰리': 65, '고': 66, '최악': 67, '고통': 68, '받을껀데': 69, '닥터스': 70, '트레인': 71, '지': 72, '판단력': 73, '좋았다': 74, '이야': 75, '전체': 76, '보는거니': 77, '받을수도': 78, '있다고': 79, '보는데': 80, '줬어도': 81, '없음': 82, '참신': 83, '성': 84, '빛': 85, '발함': 86, '결론': 87, '적': 88, '상과': 89, '받았어도': 90, '아무': 91, '없는': 92, '연출': 93, '력': 94, '만으로': 95, '최고': 96, '투명인간': 97, '되서':

In [58]:
vocab_size = 50

tokenizer = Tokenizer()
tokenizer.fit_on_texts(results)

In [59]:
vocab_size = 50

# 인덱스가 50 초과인 단어 제거
words_frequency = [w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1]
for w in words_frequency:
    del tokenizer.word_index[w]
    del tokenizer.word_counts[w]
    
print(tokenizer.word_index)
print()
print(tokenizer.word_counts)
print()
print(tokenizer.texts_to_sequences(results))

{'작품': 1, '상': 2, '감상': 3, '평': 4, '할말': 5, '감독': 6, '장면': 7, '진짜': 8, '주인공': 9, '다': 10, '더': 11, '아카데미': 12, '기생충': 13, '은': 14, '1917': 15, '솔직히': 16, '영화': 17, '듯': 18, '스포일러': 19, '포함': 20, '된': 21, '입니다': 22, '보기': 23, '원': 24, '테이크': 25, '촬영': 26, '충무로': 27, '거': 28, '어케': 29, '하는거냐': 30, '촬영감독': 31, '영혼': 32, '갈아': 33, '넣은': 34, '마스터피스': 35, '오스카': 36, '탔어도': 37, '없었을것': 38, '카메라': 39, '계속': 40, '따라가는데': 41, '세트': 42, '장이': 43, '엄청': 44, '넓은거에': 45, '놀랐습니다': 46, '직접': 47, '1': 48, '차': 49, '대전': 50}

OrderedDict([('충무로', 1), ('거', 1), ('어케', 1), ('하는거냐', 1), ('촬영감독', 1), ('영혼', 1), ('갈아', 1), ('넣은', 1), ('마스터피스', 1), ('오스카', 1), ('작품', 4), ('상', 4), ('탔어도', 1), ('할말', 3), ('없었을것', 1), ('주인공', 2), ('카메라', 1), ('계속', 1), ('따라가는데', 1), ('세트', 1), ('장이', 1), ('엄청', 1), ('넓은거에', 1), ('놀랐습니다', 1), ('직접', 1), ('1', 1), ('차', 1), ('대전', 1), ('다', 2), ('더', 2), ('아카데미', 2), ('기생충', 2), ('감독', 3), ('은', 2), ('1917', 2), ('솔직히', 2), ('영화', 2), ('듯', 2), ('장면', 3), ('스포일러', 2), ('포함', 2), 

In [60]:
vocab_size = 20

# keras는 0부터 인덱싱, 0과 1 제외 위해 '+ 2'
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(results)

print(tokenizer.texts_to_sequences(results))

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [3], [1], [6], [1], [10], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [11], [1], [1], [1], [12], [1], [1], [1], [1], [1], [1], [1], [1], [13], [2], [3], [1], [2], [1], [1], [14], [1], [1], [1], [7], [3], [15], [16], [1], [6], [1], [7], [1], [1], [1], [1], [1], [1], [13], [2], [1], [7], [3], [16], [1], [1], [6], [1], [17], [1], [1], [1], [1], [18], [11], [1], [1], [1], [19], [1], [1], [8], [15], [18], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [20], [21], [1], [4], [5], [1], [4], [5], [1], [9], [1], [1], [8], [1], [1], [1], [1], [1], [9], [1], [19], [17], [1], [14], [1], [1], [12], [1], [1], [1], [1], [1], [1], [1], [10], [1], [1], [1], [8], [9], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [20], [21], [1], [4], [5], [1], [4], [5], [1]]


- 1은 00V

## 인코딩: One-hot-Encoding

In [61]:
from tensorflow.keras.utils import to_categorical

In [62]:
encoded = tokenizer.texts_to_sequences([results][0])
print(encoded)

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [3], [1], [6], [1], [10], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [11], [1], [1], [1], [12], [1], [1], [1], [1], [1], [1], [1], [1], [13], [2], [3], [1], [2], [1], [1], [14], [1], [1], [1], [7], [3], [15], [16], [1], [6], [1], [7], [1], [1], [1], [1], [1], [1], [13], [2], [1], [7], [3], [16], [1], [1], [6], [1], [17], [1], [1], [1], [1], [18], [11], [1], [1], [1], [19], [1], [1], [8], [15], [18], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [20], [21], [1], [4], [5], [1], [4], [5], [1], [9], [1], [1], [8], [1], [1], [1], [1], [1], [9], [1], [19], [17], [1], [14], [1], [1], [12], [1], [1], [1], [1], [1], [1], [1], [10], [1], [1], [1], [8], [9], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [20], [21], [1], [4], [5], [1], [4], [5], [1]]


In [63]:
one_hot = to_categorical(encoded)

# 전체 인코딩 결과
print(one_hot)

# 한 문장에 대한 인코딩 결과
print(one_hot[0])

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
